In [1]:
import csv
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile
import pickle
import os

In [14]:
#read link data
link_path = os.path.join("transportation_data","wislr-data","wislr_rdwy_link.csv")
link_data = pd.read_csv(link_path)

#read link time
time_path = os.path.join("link_time.csv")
time_data = pd.read_csv(time_path,index_col=0)

#read link name
name_path = os.path.join("transportation_data","wislr-data","wislr_rdwy_rte_link.csv")
name_data = pd.read_csv(name_path)

In [15]:
# add road name and link id
time_data['RDWY_LINK_ID'] = link_data['RDWY_LINK_ID']


In [16]:
# add roadway rte id
RDWY_RTE_ID = []
for i in list(time_data['RDWY_LINK_ID']):
    RDWY_RTE_ID.append(name_data.loc[name_data['RDWY_LINK_ID'] == i,'RDWY_RTE_ID'].iloc[0])

time_data["RDWY_RTE_ID"] = RDWY_RTE_ID

In [17]:
time_data['FROM'] = time_data['FROM'].astype(str)
time_data['TO'] = time_data['TO'].astype(str)

In [18]:
time_data

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID
0,1661200,1660820,116.171429,4818580,400333
1,1661678,1661705,6.320000,4819008,396810
2,1661704,1661705,10.560000,4819009,396788
3,1661723,1661697,23.240000,4819014,396769
4,1661705,1661723,10.560000,4819015,396736
...,...,...,...,...,...
3127,1663210,1663276,25.075000,5559301,395130
3128,1661558,1661551,9.057143,5559302,395130
3129,1661551,1661558,9.057143,5559529,395003
3130,2016715,2005025,31.657143,5559530,395003


In [19]:
name_data.head()

,WISLR_PTY_ID,WISLR_PTY_TYCD,WISLR_PTY_NM,RDWY_RTE_ID,SORT_NM,RDWY_LINK_ID,RWLK_STRT_OSET,RWLK_END_OSET,RWRL_SORT,BASC_RTE_FL,DIR_INDC
0,333,T,CROSS PLAINS,394349,AIRPORT RD,4840537,264,528,100,Y,>
1,333,T,CROSS PLAINS,394349,AIRPORT RD,5253200,0,132,200,Y,>
2,333,T,CROSS PLAINS,394349,AIRPORT RD,5253202,197,7762,300,Y,>
3,333,T,CROSS PLAINS,394349,AIRPORT RD,5253201,0,7565,400,N,<
4,333,T,CROSS PLAINS,394349,AIRPORT RD,5253199,185,317,500,N,<


In [20]:
name_data

,WISLR_PTY_ID,WISLR_PTY_TYCD,WISLR_PTY_NM,RDWY_RTE_ID,SORT_NM,RDWY_LINK_ID,RWLK_STRT_OSET,RWLK_END_OSET,RWRL_SORT,BASC_RTE_FL,DIR_INDC
0,333,T,CROSS PLAINS,394349,AIRPORT RD,4840537,264,528,100,Y,>
1,333,T,CROSS PLAINS,394349,AIRPORT RD,5253200,0,132,200,Y,>
2,333,T,CROSS PLAINS,394349,AIRPORT RD,5253202,197,7762,300,Y,>
3,333,T,CROSS PLAINS,394349,AIRPORT RD,5253201,0,7565,400,N,<
4,333,T,CROSS PLAINS,394349,AIRPORT RD,5253199,185,317,500,N,<
...,...,...,...,...,...,...,...,...,...,...,...
3376,379,C,MIDDLETON,702113,BREAKERS ROCK RD,5422167,0,947,400,N,<
3377,379,C,MIDDLETON,744117,CROSSOVER USH 00012,5519750,0,72,100,Y,>
3378,379,C,MIDDLETON,744117,CROSSOVER USH 00012,5519749,0,72,200,N,<
3379,379,C,MIDDLETON,756816,SCHWARTZ RD,5035654,0,898,99,Y,>


In [21]:
time_data.head()

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID
0,1661200,1660820,116.171429,4818580,400333
1,1661678,1661705,6.320000,4819008,396810
2,1661704,1661705,10.560000,4819009,396788
3,1661723,1661697,23.240000,4819014,396769
4,1661705,1661723,10.560000,4819015,396736


In [36]:
arc_data = time_data.copy()
arc_data['use'] = 1
arc_data.loc[arc_data["FROM"]==arc_data["TO"] ,'use'] = 0

#change data type to string
arc_data['FROM'] = arc_data['FROM'].astype(str)
arc_data['TO'] = arc_data['TO'].astype(str)
# start processing nodes

# find all the unique nodes
nodes = set(list(arc_data["FROM"]) + list(arc_data["TO"]))
interesction_index = 1

loopTrack = 1
# for each node, start generating buffer node acccording to the in and out arc
for N in nodes:
    new_arcs = []
    out_N = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['TO'])
    out_N_time = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['TIME'])
    out_N_id = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['RDWY_LINK_ID'])
    out_N_rteid = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['RDWY_RTE_ID'])
    in_N = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['FROM'])
    in_N_time = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['TIME'])
    in_N_id = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['RDWY_LINK_ID'])
    in_N_rteid = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['RDWY_RTE_ID'])


    new_nodes = list(set(out_N + in_N))
    new_nodes_dict = {}
    for i in range(len(new_nodes)):
        new_nodes_dict[new_nodes[i]] = str(N) + "_"+ str(i + 1) 
    
    # connnect new nodes with original nodes
    for index in range(len(out_N)):
        new_arcs.append([new_nodes_dict[out_N[index]],out_N[index],out_N_time[index],out_N_id[index],\
                         out_N_rteid[index],1])
        
    for index in range(len(in_N)):
        new_arcs.append([in_N[index],new_nodes_dict[in_N[index]],in_N_time[index],in_N_id[index],\
                         in_N_rteid[index],1])
        
    # connect new nodes with each other           
    for i in in_N:
        for o in out_N:
            # do not allow turn around
            if i != o:
                #turn
                if in_N_rteid[in_N.index(i)] != out_N_rteid[out_N.index(o)]:
                    new_arcs.append([new_nodes_dict[i],new_nodes_dict[o],-1,'intersection_'+str(interesction_index),\
                                 'intersection_'+str(interesction_index),1])
                #go straight
                else:
                    new_arcs.append([new_nodes_dict[i],new_nodes_dict[o],-2,'intersection_'+str(interesction_index),\
                                 'intersection_'+str(interesction_index),1])
                interesction_index += 1
                
    # disconnect original arcs
    arc_data.loc[(arc_data['FROM'] == N) | (arc_data['TO'] == N) , 'use'] = 0
    
    # add arcs to arc set
    new_arcs = pd.DataFrame(new_arcs, columns=["FROM","TO","TIME","RDWY_LINK_ID","RDWY_RTE_ID","use"])
    arc_data = arc_data.append(new_arcs,ignore_index=True)
    

# connect alias node according to in and out arcs
    # take note of the original link id and traveling time
    # take note of the length of each link
    
arc_data = arc_data[arc_data['use'] == 1]

arc_data = arc_data.reset_index(drop=True)

In [39]:
arc_data

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID,use
0,1662351_2,1662351_3,-1.0,intersection_1,intersection_1,1
1,1662351_2,1662351_1,-1.0,intersection_2,intersection_2,1
2,1662351_3,1662351_1,-1.0,intersection_3,intersection_3,1
3,1662351_3,1662351_2,-1.0,intersection_4,intersection_4,1
4,1662351_1,1662351_3,-1.0,intersection_5,intersection_5,1
...,...,...,...,...,...,...
9037,1664797_1,1664797_2,-1.0,intersection_5912,intersection_5912,1
9038,1664797_2,1664797_3,-1.0,intersection_5913,intersection_5913,1
9039,1664797_2,1664797_1,-1.0,intersection_5914,intersection_5914,1
9040,2020338_1,1663605_3,6.6,5049298,400416,1


In [49]:
arc_data["TRAVEL_TIME"] =  arc_data["TIME"] 
arc_data["TURN"] =  0
arc_data.loc[arc_data['TIME'] < 0,"TRAVEL_TIME"] = 0
arc_data.loc[arc_data['TIME'] == -1,"TURN"] = 1

In [50]:
arc_data

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID,use,TRAVEL_TIME,TURN
0,1662351_2,1662351_3,-1.0,intersection_1,intersection_1,1,0.0,1
1,1662351_2,1662351_1,-1.0,intersection_2,intersection_2,1,0.0,1
2,1662351_3,1662351_1,-1.0,intersection_3,intersection_3,1,0.0,1
3,1662351_3,1662351_2,-1.0,intersection_4,intersection_4,1,0.0,1
4,1662351_1,1662351_3,-1.0,intersection_5,intersection_5,1,0.0,1
...,...,...,...,...,...,...,...,...
9037,1664797_1,1664797_2,-1.0,intersection_5912,intersection_5912,1,0.0,1
9038,1664797_2,1664797_3,-1.0,intersection_5913,intersection_5913,1,0.0,1
9039,1664797_2,1664797_1,-1.0,intersection_5914,intersection_5914,1,0.0,1
9040,2020338_1,1663605_3,6.6,5049298,400416,1,6.6,0
